# Neural Network

Now I'm going to implement a neural net. I'll use keras and tensorflow to do it.

In [17]:
import numpy as np
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import StratifiedKFold, train_test_split
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from keras.callbacks import *
from keras.layers import Dense, Dropout, Input, Activation, BatchNormalization
from keras.models import Sequential, Model, load_model
from keras.wrappers.scikit_learn import KerasClassifier
import warnings
#warnings.filterwarnings('ignore')

In [2]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod(
            (datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' %
              (thour, tmin, round(tsec, 2)))

In [3]:
# Importing utils 
os.chdir('C:\\Users\\hugo_\\OneDrive\\Documentos\\DataScience\\Repos\\kaggle_credit_risk\\code')

from utils import *

# Data directory
os.chdir('C:\\Users\\hugo_\\OneDrive\\Documentos\\DataScience\\Repos\\kaggle_credit_risk\\data\\treated')

In [4]:
train = pd.read_csv('train_eng_linear.csv')
test = pd.read_csv('test_eng_linear.csv')

In [5]:
ID_train = train.SK_ID_CURR.values
ID_test = test.SK_ID_CURR.values

y_train = train.TARGET.values

X_train = train.drop(['SK_ID_CURR', 'TARGET'], axis=1).values
X_test = test.drop(['SK_ID_CURR'], axis=1).values

Since our neural network is a linear model, it is necessary to normalize out data. So I'm gonna use sklearn's StandardScaler to do that.

In [6]:
sc = StandardScaler().fit(X_train)
X_train_scaled = sc.transform(X_train)
X_test_scaled = sc.transform(X_test)

In [7]:
X_train_scaled.shape

(307511, 278)

Let's algo create a validation set, which I'm going to use 20% of our training data as holdout data.

In [ ]:
X_train_, X_val_, y_train_, y_val_ = train_test_split(X_train_scaled, y_train, test_size = 0.2, random_state = 42)

Let's now create our neural net. First, I'm gonna use keras with tensorflow as backend.

I just want to compare performances of both libs, in terms of performance (AUC score) and processing time.

# Keras + Tensorflow

First, let's define a AUC callback. This is necessary in order to follow up the scores on the end of each batch during training time. AUC is not a built-in metric on keras, so it is necessary to use the base class Callback and create this class.

In [24]:
class AUC_Callback(Callback):
    
    def __init__(self, X_train, y_train, X_val, y_val):
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        
    def on_train_begin(self, logs={}):
        self.aucs = []
        self.losses = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        self.losses.append(logs.get('loss'))
        
        # Calculating auc on train
        y_pred_train = self.model.predict_proba(self.X_train)
        print(np.unique(self.y_train))
        auc_train = roc_auc_score(self.y_train, y_pred_train)
        logs['auc_train'] = auc_train
        
        # Calculating auc on val
        y_pred = self.model.predict_proba(self.X_val)
        auc_val = roc_auc_score(self.y_val, y_pred)
        logs['auc_val'] = auc_val
        
        print('\rauc_train: %s - auc_val: %s' % (str(round(auc_train,5)), str(round(auc_val,5))))
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

Then, I'm going to create a function which builds and returns our neural net. This can be use later on a sklearn wrapper class called KerasClassifier, which takes this function as a building block into its API. 

Ok, let's define a few things. First, and AUC score must be defined for the network. It is done by using tensorflow's metrics. Then, I'm defining a few folds to run cross validation, and a number of runs, in which I'm gonna average over my neural nets. 

I'll define a build_nn function, which returns a Keras neural net model in order to feed a KerasClassifier, a sklearn's wrapper that takes a building function to construct our classifier. 

A list of callbacks is passed, our AUC_callback class, early stopping, which will terminate the training if the score doesn't improve in a few runs, and a model checkpoint, which is going to save the best model along the epochs.

In [35]:
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [36]:
folds = 5
runs = 5
cv_AUC = 0

fpred = []
avpred = []
avreal = []
avids = []

In [42]:
os.chdir('C:\\Users\\hugo_\\OneDrive\\Documentos\\DataScience\\Repos\\kaggle_credit_risk\\checkpoints')

# K-fold cross validation
skf = StratifiedKFold(n_splits=folds, random_state = 1234)
starttime = timer(None)

# This is the main loop, where I iterate through the data splits
for i, (train_index, test_index) in enumerate(skf.split(X_train_scaled, y_train)):
    start_time = timer(None)
    X_train_, X_val_ = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_, y_val_ = y_train[train_index], y_train[test_index]
    train_ids, val_ids = ID_train[train_index], ID_train[test_index]
    

    # This definition must be within the for loop or else it will continue training previous model
    def build_nn():
        model = Sequential()
        model.add(Dense(200, input_dim=X_train_.shape[1], kernel_initializer='glorot_normal'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(100, kernel_initializer='glorot_normal'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))
        model.add(Dense(50, kernel_initializer='glorot_normal'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.15))
        model.add(Dense(25, kernel_initializer='glorot_normal'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.1))
        model.add(Dense(1, activation='sigmoid'))

        # Compile model
        model.compile(optimizer='adam', metrics = [auc], loss='binary_crossentropy')

        return model

    # This is the loop in the runs where I'm gonna average over later on
    for run in range(runs):
        print('\n Fold %d - Run %d\n' % ((i + 1), (run + 1)))
        np.random.seed()

        # Defining callbacks for our keras model. AUC_Callback to calculate auc, Early Stopping in case the is
        # no improvment on scores after a few iterations, and model checkpoint to save the best possible model. 
        callbacks = [
            AUC_Callback(X_train = X_train_, y_train = y_train_, X_val = X_val_, y_val = y_val_),
            EarlyStopping(monitor='val_auc', patience=10, mode='max', verbose=1),
            ModelCheckpoint(
                    'keras-5fold-run-01-v1-fold-' + str('%02d' % (i + 1)) + '-run-' + str('%02d' % (run + 1)) + '.check',
                    monitor='val_auc', mode='max', # mode must be set to max or Keras will be confused
                    save_best_only=True,
                    verbose=1)
        ]

        # Defining the classifier on keras sklearn API
        nnet = KerasClassifier(
            build_fn=build_nn,
            epochs=200,
            batch_size=1500,
            validation_data=(X_val_, y_val_),
            verbose=2,
            shuffle=True,
            callbacks=callbacks)

        # FItting the model
        fit = nnet.fit(X_train_, y_train_)

        # Garbage collector to save some memory
        del nnet
        
        # Loading best model and predicting on validation/test data
        nnet = load_model('keras-5fold-run-01-v1-fold-' + str('%02d' % (i + 1)) + '-run-' + str('%02d' % (run + 1)) + '.check',
                         custom_objects={"auc": auc})
        
        # Predicting on validation set
        y_pred_val_run = nnet.predict_proba(X_val_, verbose=0)
        
        # Calculating AUC on currnt run
        AUC_run = roc_auc_score(y_val_, y_pred_val_run)
        print(' Fold %d Run %d AUC: %.5f' % ((i + 1), (run + 1), AUC_run))
        
        # Predicting on test set, I'll average it later
        y_pred_run = nnet.predict_proba(X_test_scaled, verbose=0)
        
        
        if run > 0:
            y_pred_val = y_pred_val + y_pred_val_run
            y_pred = y_pred + y_pred_run
        else:
            y_pred_val = y_pred_val_run
            y_pred = y_pred_run
            
# We average all runs from the same fold and provide a parameter summary for each fold. Unless something 
# is wrong, the numbers printed here should be better than any of the individual runs.

    y_pred_val = y_pred_val / runs
    y_pred = y_pred / runs
    AUC = roc_auc_score(y_val_, y_pred_val)
    print(' Fold %d AUC: %.5f' % ((i + 1), AUC))
    timer(start_time)
    
# We add up predictions on the test data for each fold. Create out-of-fold predictions for validation data.

    if i > 0:
        fpred = pred + y_pred
        avreal = np.concatenate((avreal, y_val_), axis=0)
        avpred = np.concatenate((avpred, y_pred_val), axis=0)
        avids = np.concatenate((avids, val_ids), axis=0)
    else:
        fpred = y_pred
        avreal = y_val_
        avpred = y_pred_val
        avids = val_ids
    pred = fpred
    cv_AUC = cv_AUC + AUC

mpred = pred / folds


 Fold 1 - Run 1

Train on 205007 samples, validate on 102504 samples
Epoch 1/200
 - 29s - loss: 0.5831 - auc: 0.5624 - val_loss: 0.4239 - val_auc: 0.5977
[0 1]
auc_train: 0.70882 - auc_val: 0.70174

Epoch 00001: val_auc improved from -inf to 0.59771, saving model to keras-5fold-run-01-v1-fold-01-run-01.check
Epoch 2/200
 - 17s - loss: 0.3273 - auc: 0.6051 - val_loss: 0.2735 - val_auc: 0.6000
[0 1]
auc_train: 0.72975 - auc_val: 0.72353

Epoch 00002: val_auc improved from 0.59771 to 0.60001, saving model to keras-5fold-run-01-v1-fold-01-run-01.check
Epoch 3/200
 - 17s - loss: 0.2701 - auc: 0.6052 - val_loss: 0.2577 - val_auc: 0.6112
[0 1]
auc_train: 0.74009 - auc_val: 0.73321

Epoch 00003: val_auc improved from 0.60001 to 0.61116, saving model to keras-5fold-run-01-v1-fold-01-run-01.check
Epoch 4/200
 - 18s - loss: 0.2618 - auc: 0.6190 - val_loss: 0.2548 - val_auc: 0.6259
[0 1]
auc_train: 0.74678 - auc_val: 0.73794

Epoch 00004: val_auc improved from 0.61116 to 0.62590, saving model to 

[0 1]
auc_train: 0.8065 - auc_val: 0.74681

Epoch 00034: val_auc improved from 0.73391 to 0.73494, saving model to keras-5fold-run-01-v1-fold-01-run-01.check
Epoch 35/200
 - 17s - loss: 0.2404 - auc: 0.7354 - val_loss: 0.2505 - val_auc: 0.7359
[0 1]
auc_train: 0.80817 - auc_val: 0.74748

Epoch 00035: val_auc improved from 0.73494 to 0.73592, saving model to keras-5fold-run-01-v1-fold-01-run-01.check
Epoch 36/200
 - 19s - loss: 0.2397 - auc: 0.7364 - val_loss: 0.2510 - val_auc: 0.7369
[0 1]
auc_train: 0.81075 - auc_val: 0.74615

Epoch 00036: val_auc improved from 0.73592 to 0.73688, saving model to keras-5fold-run-01-v1-fold-01-run-01.check
Epoch 37/200
 - 17s - loss: 0.2396 - auc: 0.7374 - val_loss: 0.2512 - val_auc: 0.7378
[0 1]
auc_train: 0.81321 - auc_val: 0.74532

Epoch 00037: val_auc improved from 0.73688 to 0.73779, saving model to keras-5fold-run-01-v1-fold-01-run-01.check
Epoch 38/200
 - 19s - loss: 0.2390 - auc: 0.7383 - val_loss: 0.2514 - val_auc: 0.7387
[0 1]
auc_train: 0.81

 - 20s - loss: 0.2289 - auc: 0.7567 - val_loss: 0.2543 - val_auc: 0.7569
[0 1]
auc_train: 0.84861 - auc_val: 0.73773

Epoch 00068: val_auc improved from 0.75647 to 0.75692, saving model to keras-5fold-run-01-v1-fold-01-run-01.check
Epoch 69/200
 - 19s - loss: 0.2291 - auc: 0.7572 - val_loss: 0.2544 - val_auc: 0.7574
[0 1]
auc_train: 0.85118 - auc_val: 0.73644

Epoch 00069: val_auc improved from 0.75692 to 0.75735, saving model to keras-5fold-run-01-v1-fold-01-run-01.check
Epoch 70/200
 - 17s - loss: 0.2294 - auc: 0.7575 - val_loss: 0.2546 - val_auc: 0.7577
[0 1]
auc_train: 0.85117 - auc_val: 0.73631

Epoch 00070: val_auc improved from 0.75735 to 0.75775, saving model to keras-5fold-run-01-v1-fold-01-run-01.check
Epoch 71/200
 - 19s - loss: 0.2288 - auc: 0.7580 - val_loss: 0.2553 - val_auc: 0.7582
[0 1]
auc_train: 0.85087 - auc_val: 0.73541

Epoch 00071: val_auc improved from 0.75775 to 0.75816, saving model to keras-5fold-run-01-v1-fold-01-run-01.check
Epoch 72/200
 - 17s - loss: 0.228

Epoch 102/200
 - 19s - loss: 0.2223 - auc: 0.7683 - val_loss: 0.2577 - val_auc: 0.7684
[0 1]
auc_train: 0.87581 - auc_val: 0.72894

Epoch 00102: val_auc improved from 0.76812 to 0.76839, saving model to keras-5fold-run-01-v1-fold-01-run-01.check
Epoch 103/200
 - 18s - loss: 0.2221 - auc: 0.7685 - val_loss: 0.2580 - val_auc: 0.7686
[0 1]
auc_train: 0.87291 - auc_val: 0.73084

Epoch 00103: val_auc improved from 0.76839 to 0.76865, saving model to keras-5fold-run-01-v1-fold-01-run-01.check
Epoch 104/200
 - 18s - loss: 0.2219 - auc: 0.7688 - val_loss: 0.2578 - val_auc: 0.7689
[0 1]
auc_train: 0.87642 - auc_val: 0.72946

Epoch 00104: val_auc improved from 0.76865 to 0.76891, saving model to keras-5fold-run-01-v1-fold-01-run-01.check
Epoch 105/200
 - 18s - loss: 0.2218 - auc: 0.7690 - val_loss: 0.2580 - val_auc: 0.7692
[0 1]
auc_train: 0.87552 - auc_val: 0.73016

Epoch 00105: val_auc improved from 0.76891 to 0.76917, saving model to keras-5fold-run-01-v1-fold-01-run-01.check
Epoch 106/200
 -

[0 1]
auc_train: 0.89034 - auc_val: 0.722

Epoch 00135: val_auc improved from 0.77569 to 0.77588, saving model to keras-5fold-run-01-v1-fold-01-run-01.check
Epoch 136/200
 - 19s - loss: 0.2163 - auc: 0.7760 - val_loss: 0.2607 - val_auc: 0.7761
[0 1]
auc_train: 0.89153 - auc_val: 0.72047

Epoch 00136: val_auc improved from 0.77588 to 0.77607, saving model to keras-5fold-run-01-v1-fold-01-run-01.check
Epoch 137/200
 - 19s - loss: 0.2165 - auc: 0.7762 - val_loss: 0.2617 - val_auc: 0.7763
[0 1]
auc_train: 0.88998 - auc_val: 0.72442

Epoch 00137: val_auc improved from 0.77607 to 0.77625, saving model to keras-5fold-run-01-v1-fold-01-run-01.check
Epoch 138/200
 - 19s - loss: 0.2168 - auc: 0.7763 - val_loss: 0.2610 - val_auc: 0.7764
[0 1]
auc_train: 0.89239 - auc_val: 0.72103

Epoch 00138: val_auc improved from 0.77625 to 0.77643, saving model to keras-5fold-run-01-v1-fold-01-run-01.check
Epoch 139/200
 - 20s - loss: 0.2157 - auc: 0.7765 - val_loss: 0.2609 - val_auc: 0.7766
[0 1]
auc_train: 0

...

Epoch 44/200
 - 23s - loss: 0.2373 - auc: 0.7442 - val_loss: 0.2493 - val_auc: 0.7446
[0 1]
auc_train: 0.82214 - auc_val: 0.75081

Epoch 00044: val_auc improved from 0.74385 to 0.74459, saving model to keras-5fold-run-01-v1-fold-03-run-02.check
Epoch 45/200
 - 24s - loss: 0.2370 - auc: 0.7450 - val_loss: 0.2494 - val_auc: 0.7453
[0 1]
auc_train: 0.82354 - auc_val: 0.75032

Epoch 00045: val_auc improved from 0.74459 to 0.74529, saving model to keras-5fold-run-01-v1-fold-03-run-02.check
Epoch 46/200
 - 25s - loss: 0.2365 - auc: 0.7457 - val_loss: 0.2497 - val_auc: 0.7460
[0 1]
auc_train: 0.82323 - auc_val: 0.75006

Epoch 00046: val_auc improved from 0.74529 to 0.74599, saving model to keras-5fold-run-01-v1-fold-03-run-02.check
Epoch 47/200
 - 24s - loss: 0.2360 - auc: 0.7464 - val_loss: 0.2495 - val_auc: 0.7467
[0 1]
auc_train: 0.82527 - auc_val: 0.75035

Epoch 00047: val_auc improved from 0.74599 to 0.74669, saving model to keras-5fold-run-01-v1-fold-03-run-02.check
Epoch 48/200
 - 23s 

 - 20s - loss: 0.2280 - auc: 0.7619 - val_loss: 0.2530 - val_auc: 0.7621
[0 1]
auc_train: 0.85289 - auc_val: 0.74375

Epoch 00078: val_auc improved from 0.76171 to 0.76207, saving model to keras-5fold-run-01-v1-fold-03-run-02.check
Epoch 79/200
 - 20s - loss: 0.2273 - auc: 0.7623 - val_loss: 0.2530 - val_auc: 0.7624
[0 1]
auc_train: 0.8546 - auc_val: 0.74173

Epoch 00079: val_auc improved from 0.76207 to 0.76245, saving model to keras-5fold-run-01-v1-fold-03-run-02.check
Epoch 80/200
 - 20s - loss: 0.2271 - auc: 0.7626 - val_loss: 0.2526 - val_auc: 0.7628
[0 1]
auc_train: 0.85552 - auc_val: 0.74257

Epoch 00080: val_auc improved from 0.76245 to 0.76280, saving model to keras-5fold-run-01-v1-fold-03-run-02.check
Epoch 81/200
 - 20s - loss: 0.2270 - auc: 0.7630 - val_loss: 0.2539 - val_auc: 0.7632
[0 1]
auc_train: 0.85806 - auc_val: 0.74083

Epoch 00081: val_auc improved from 0.76280 to 0.76316, saving model to keras-5fold-run-01-v1-fold-03-run-02.check
Epoch 82/200
 - 20s - loss: 0.2271

Epoch 112/200
 - 20s - loss: 0.2206 - auc: 0.7720 - val_loss: 0.2565 - val_auc: 0.7721
[0 1]
auc_train: 0.87784 - auc_val: 0.73536

Epoch 00112: val_auc improved from 0.77190 to 0.77214, saving model to keras-5fold-run-01-v1-fold-03-run-02.check
Epoch 113/200
 - 20s - loss: 0.2202 - auc: 0.7723 - val_loss: 0.2560 - val_auc: 0.7724
[0 1]
auc_train: 0.87766 - auc_val: 0.73475

Epoch 00113: val_auc improved from 0.77214 to 0.77239, saving model to keras-5fold-run-01-v1-fold-03-run-02.check
Epoch 114/200
 - 20s - loss: 0.2205 - auc: 0.7725 - val_loss: 0.2564 - val_auc: 0.7726
[0 1]
auc_train: 0.87915 - auc_val: 0.73292

Epoch 00114: val_auc improved from 0.77239 to 0.77263, saving model to keras-5fold-run-01-v1-fold-03-run-02.check
Epoch 115/200
 - 20s - loss: 0.2192 - auc: 0.7728 - val_loss: 0.2571 - val_auc: 0.7729
[0 1]
auc_train: 0.88075 - auc_val: 0.73127

Epoch 00115: val_auc improved from 0.77263 to 0.77287, saving model to keras-5fold-run-01-v1-fold-03-run-02.check
Epoch 116/200
 -

[0 1]
auc_train: 0.89275 - auc_val: 0.72885

Epoch 00145: val_auc improved from 0.77880 to 0.77899, saving model to keras-5fold-run-01-v1-fold-03-run-02.check
Epoch 146/200
 - 20s - loss: 0.2156 - auc: 0.7791 - val_loss: 0.2591 - val_auc: 0.7792
[0 1]
auc_train: 0.89119 - auc_val: 0.73045

Epoch 00146: val_auc improved from 0.77899 to 0.77917, saving model to keras-5fold-run-01-v1-fold-03-run-02.check
Epoch 147/200
 - 20s - loss: 0.2149 - auc: 0.7793 - val_loss: 0.2596 - val_auc: 0.7794
[0 1]
auc_train: 0.89461 - auc_val: 0.72648

Epoch 00147: val_auc improved from 0.77917 to 0.77935, saving model to keras-5fold-run-01-v1-fold-03-run-02.check
Epoch 148/200
 - 20s - loss: 0.2153 - auc: 0.7794 - val_loss: 0.2595 - val_auc: 0.7795
[0 1]
auc_train: 0.89229 - auc_val: 0.72908

Epoch 00148: val_auc improved from 0.77935 to 0.77953, saving model to keras-5fold-run-01-v1-fold-03-run-02.check
Epoch 149/200
 - 20s - loss: 0.2153 - auc: 0.7796 - val_loss: 0.2594 - val_auc: 0.7797
[0 1]
auc_train:

Epoch 179/200
 - 20s - loss: 0.2116 - auc: 0.7843 - val_loss: 0.2613 - val_auc: 0.7843
[0 1]
auc_train: 0.90476 - auc_val: 0.72251

Epoch 00179: val_auc improved from 0.78420 to 0.78433, saving model to keras-5fold-run-01-v1-fold-03-run-02.check
Epoch 180/200
 - 20s - loss: 0.2111 - auc: 0.7844 - val_loss: 0.2604 - val_auc: 0.7845
[0 1]
auc_train: 0.90376 - auc_val: 0.72555

Epoch 00180: val_auc improved from 0.78433 to 0.78447, saving model to keras-5fold-run-01-v1-fold-03-run-02.check
Epoch 181/200
 - 20s - loss: 0.2113 - auc: 0.7845 - val_loss: 0.2604 - val_auc: 0.7846
[0 1]
auc_train: 0.90296 - auc_val: 0.72485

Epoch 00181: val_auc improved from 0.78447 to 0.78461, saving model to keras-5fold-run-01-v1-fold-03-run-02.check
Epoch 182/200
 - 20s - loss: 0.2104 - auc: 0.7847 - val_loss: 0.2604 - val_auc: 0.7848
[0 1]
auc_train: 0.90406 - auc_val: 0.725

Epoch 00182: val_auc improved from 0.78461 to 0.78476, saving model to keras-5fold-run-01-v1-fold-03-run-02.check
Epoch 183/200
 - 2

In [47]:
mpred.flatten()

array([0.03799157, 0.22590744, 0.06207643, ..., 0.05695406, 0.08194356,
       0.17112613], dtype=float32)

In [48]:
# creating submission file
pd.DataFrame(
    {
        'SK_ID_CURR': ID_test,
        'TARGET': mpred.flatten()
    }
).to_csv('submission_nnet.csv', index = None)

This submission got __0.73423__ on leaderboard. It's not so good as tree based models, but it has a lot of room for improvement. Unfortunatelly, it's quite expensive to run neural nets, but in general it scores good results.